In [1]:
import openai, json

In [2]:
from pptx import Presentation

In [4]:
presentation_title = input('What do you want to make a presentation about?')

What do you want to make a presentation about? climate change


In [5]:
query_json = """{
    "input_text": "[[QUERY]]",
    "output_format": "json",
    "json_structure": {
        "slides" : "{{presentation_slides}}"
        }
        }"""

In [6]:
question = "You are a powerpoint presentation specialist, skilled in creating clear,consise, and engaging slide content. Your task is to generate content for a presentation about "+ presentation_title + "Each slide must have a {{header}} and {{content}}. Return as JSON array, where each object in the array represents a slide. Do not include any introductory text or explanation outside JSON. " + "For each slide, include: a header: the title of the slide, and content: An array of bullet points or paragraphs for the main text. Each bullet point must be concise. Limit each slide to a maximum of 5 bullet points. The first slide must be a title slide with a subtitle field"
question

'You are a powerpoint presentation specialist, skilled in creating clear,consise, and engaging slide content. Your task is to generate content for a presentation about climate changeEach slide must have a {{header}} and {{content}}. Return as JSON array, where each object in the array represents a slide. Do not include any introductory text or explanation outside JSON. For each slide, include: a header: the title of the slide, and content: An array of bullet points or paragraphs for the main text. Each bullet point must be concise. Limit each slide to a maximum of 5 bullet points. The first slide must be a title slide with a subtitle field'

In [7]:
prompt = query_json.replace("[[QUERY]]",question)
print(prompt)

{
    "input_text": "You are a powerpoint presentation specialist, skilled in creating clear,consise, and engaging slide content. Your task is to generate content for a presentation about climate changeEach slide must have a {{header}} and {{content}}. Return as JSON array, where each object in the array represents a slide. Do not include any introductory text or explanation outside JSON. For each slide, include: a header: the title of the slide, and content: An array of bullet points or paragraphs for the main text. Each bullet point must be concise. Limit each slide to a maximum of 5 bullet points. The first slide must be a title slide with a subtitle field",
    "output_format": "json",
    "json_structure": {
        "slides" : "{{presentation_slides}}"
        }
        }


In [8]:
from openai import OpenAI

In [ ]:
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key = "YOUR API KEY")
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ]
)


In [10]:
response = completion.choices[0].message.content

print(response)

{
    "slides": [
        {
            "header": "Title Slide",
            "content": [
                "Introduction to Climate Change"
            ]
        },
        {
            "header": "What is Climate Change?",
            "content": [
                "Refers to long-term changes in temperatures and weather patterns",
                "Caused by human activities such as burning fossil fuels and deforestation",
                "Leads to global warming and severe weather events"
            ]
        },
        {
            "header": "Effects of Climate Change",
            "content": [
                "Rising sea levels due to melting ice caps",
                "Increased frequency and intensity of natural disasters",
                "Disruption of ecosystems and loss of biodiversity"
            ]
        },
        {
            "header": "Mitigation Strategies",
            "content": [
                "Transition to renewable energy sources",
                "Reducing ca

In [11]:
r = json.loads(response)

In [12]:
slide_data = r["slides"]

In [13]:
prs = Presentation()

In [14]:
for slide in slide_data:
    slide_layout = prs.slide_layouts[1]
    new_slide = prs.slides.add_slide(slide_layout)
    
    if slide['header']:
        title = new_slide.shapes.title
        title.text = slide['header']
    
    if slide['content']:
        shapes = new_slide.shapes
        body_shape = shapes.placeholders[1]
        tf = body_shape.text_frame
        #tf.text = slide['content']
        #tf.fit_text(font_family="Arial", max_size=18, bold=True)
        for item in slide['content']:
            p = tf.add_paragraph()
            p.text = str(item)
            p.level = 0  # 0 = top-level bullet, 1 = sub-bullet, etc.

prs.save("output.pptx")